# Data Preparation

First let us import the required packages and modules

In [6]:
# Import the Packages

import pandas as pd
import sqlite3
import datetime


In [7]:
# Database Factory Class

class Db():
    
    def __init__(self):
        self.market_average_growth = 0
        self.database = "indian_stock_data.db"
        self.databse_connect()
        self.databse_init()
        
    def databse_connect(self):
        """Connect to the SQLite3 database."""
        self.connection = sqlite3.connect(self.database)
        self.cursor = self.connection.cursor()
    
    def databse_init(self):
        """Initialise the SQLite3 database."""
        self.cursor.execute("CREATE TABLE IF NOT EXISTS company (\
                            symbol TEXT UNIQUE NOT NULL, name TEXT)")
        self.cursor.execute("CREATE TABLE IF NOT EXISTS stock_prices (\
                            symbol TEXT NOT NULL references company(symbol) ON DELETE CASCADE, \
                            date TEXT, \
                            open REAL, high REAL, low REAL, close REAL, adj_close REAL, volume REAL, \
                            CONSTRAINT unq UNIQUE (symbol, date))")
    
    def find_companies_list(self):
        self.cursor.execute("SELECT symbol FROM company")         
        companies = [comp[0] for comp in self.cursor.fetchall()]
        return companies
    
    def insert_company_data(self, symbol, name):
        """Insert the company data to db"""
        try:
            self.cursor.execute("INSERT INTO company (symbol, name) \
                                VALUES (:symbol, :name)", \
                                {"symbol": symbol, "name": name})
            self.connection.commit()
        except sqlite3.IntegrityError as e:
            pass
    
    def insert_stock_price_data(self, symbol, date, open_, high, low, close, adj_close, volume):
        """Insert the stock data to db"""
        try:
            self.cursor.execute("INSERT INTO stock_prices (symbol, date, open, high, low, close, adj_close, volume) \
                                VALUES (:symbol, :date, :open, :high, :low, :close, :adj_close, :volume)", \
                                {"symbol": symbol, "date": date, "open": open_, "high": high, "low": low, "close": close, "adj_close": adj_close, "volume": volume})
            self.connection.commit()
        except sqlite3.IntegrityError as e:
            pass
    
    def get_stock_price_data_frame(self, start_date):
        """ Get the stock data starting from a given date and return it as a data frame"""
        self.cursor.execute("SELECT symbol, date, adj_close FROM stock_prices WHERE date >= :start_date ", {"start_date": start_date})       
        stock_prices = self.cursor.fetchall()
        stocke_prices_df = pd.DataFrame(stock_prices, columns= ['symbol', 'date', 'adj_close'])
        return stocke_prices_df
    
    def find_already_updated_stocks_list(self):
        self.cursor.execute("SELECT DISTINCT symbol FROM stock_prices")  
        companies = [comp[0] for comp in self.cursor.fetchall()]
        return companies
    
    def close(self):
        self.connection.close()

In [8]:
db = Db()

### Get the list of companies
Dowload the latest companies list from the nse website.

In [9]:
nse_companies_url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
companies_df = pd.read_csv(nse_companies_url)
companies_df.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,EQ,03-MAY-1995,10,1,INE253B01015,10
2,3IINFOTECH,3i Infotech Limited,BE,22-APR-2005,10,1,INE748C01020,10
3,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10
4,3PLAND,3P Land Holdings Limited,EQ,19-JUL-1995,2,1,INE105C01023,2


For the ease of processing we will move the data to a sqlite database.

In [10]:
def insert_company_names_to_db(companies_df):    
    for i in range(len(companies_df)) : 
        symbol = companies_df.loc[i, "SYMBOL"]
        name = companies_df.loc[i, "NAME OF COMPANY"]
        db.insert_company_data(symbol, name)
insert_company_names_to_db(companies_df)

### Download the historical stock prices

   Now let us start downloading the historical stock prices. There are bhavcopies data available at nse website, but the prices were not adjusted to the stock splits and all. So we will download the data from yahoo finance. Here we consider only nse stocks.

   Then the prices data is to be stored in the database. We are interested only in the adjusted close price value. But we will store all the data into the database for the future purposes.

In [11]:
def string_to_date(date_string):
    return datetime.datetime.strptime(date_string, '%Y-%m-%d').date()

In [12]:
symbols_already_updated = db.find_already_updated_stocks_list()
def download_historical_stock_data(stock_symbols):
    for symbol in stock_symbols:
        if symbol in symbols_already_updated:
            continue
        print(symbol)
        try:
            download_url = "https://query1.finance.yahoo.com/v7/finance/download/" + symbol + ".NS?period1=1029196800&period2=1623369600&interval=1d&events=history&includeAdjustedClose=true"
            stock_price_df = pd.read_csv(download_url)
            for i in range(len(stock_price_df)) : 
                date = string_to_date(stock_price_df.loc[i, "Date"])
                open_ = float(stock_price_df.loc[i, "Open"])
                high = float(stock_price_df.loc[i, "High"])
                low = float(stock_price_df.loc[i, "Low"])
                close = float(stock_price_df.loc[i, "Close"])
                adj_close = float(stock_price_df.loc[i, "Adj Close"])
                volume = float(stock_price_df.loc[i, "Volume"])
                db.insert_stock_price_data(symbol, date, open_, high, low, close, adj_close, volume)
        except :
            print("Error in :" , symbol)
            continue

stock_symbols_list = db.find_companies_list()
download_historical_stock_data(stock_symbols_list)
        

### Calculate the percentage of change

Here the idea is to show the percentage change of the stock prices from a start date (As the yahoo finance provides data from 2002-08-13 we took it as the start date). For this we need rearrange the stock prices information and calculate the percentage changes

In [13]:
# Get the stock price data starting from a given date(symbol and adjusted close)

start_date = datetime.datetime(2011,1,1).date()
stock_price_df = db.get_stock_price_data_frame(start_date)
stock_price_df.head()

,symbol,date,adj_close
0,20MICRONS,2011-01-03,20.874264
1,20MICRONS,2011-01-04,20.605785
2,20MICRONS,2011-01-05,21.366478
3,20MICRONS,2011-01-06,21.299356
4,20MICRONS,2011-01-07,20.605785


In [14]:
#For calculating the percentage change from a start date, it is better to pivot the table

stock_price_reshaped_df = stock_price_df.pivot(index='date', columns='symbol', values='adj_close')
stock_price_reshaped_df.head()

symbol,20MICRONS,21STCENMGM,3IINFOTECH,3MINDIA,3PLAND,5PAISA,63MOONS,A2ZINFRA,AAKASH,AARON,...,ZEEMEDIA,ZENITHEXPO,ZENSARTECH,ZENTEC,ZODIACLOTH,ZODJRDMKJ,ZOTA,ZUARI,ZUARIGLOB,ZYDUSWELL
date,,,,,,,,,,,,,,,,,,,,,
2011-01-03,20.874264,NaN,60.227486,3668.550049,23.150000,NaN,660.464905,324.341858,NaN,NaN,...,12.994885,59.450001,26.483582,NaN,243.470047,21.939245,NaN,NaN,605.238892,549.147522
2011-01-04,20.605785,NaN,59.985413,3621.100098,23.299999,NaN,656.608643,332.868042,NaN,NaN,...,13.133621,56.500000,26.350697,NaN,240.210464,22.070351,NaN,NaN,608.066711,571.197815
2011-01-05,21.366478,NaN,60.082241,3617.000000,22.799999,NaN,651.788330,334.404724,NaN,NaN,...,12.902395,56.500000,27.085485,NaN,243.470047,22.026649,NaN,NaN,591.015259,561.367004
2011-01-06,21.299356,NaN,59.646511,3650.500000,22.400000,NaN,641.739624,323.846130,NaN,NaN,...,12.717416,56.500000,27.796824,NaN,239.938873,21.152576,NaN,NaN,576.705811,563.250549
2011-01-07,20.605785,NaN,56.886890,3629.149902,22.200001,NaN,618.008545,311.701294,NaN,NaN,...,12.439945,56.500000,27.140209,NaN,230.220596,21.764425,NaN,NaN,569.979492,555.395020


In [15]:
"""
Calculate the percentage change from the previous day's closing value and 
find the cumulative product over the data frame.
"""
percentage_change_df = (stock_price_reshaped_df.pct_change()+1).cumprod()
percentage_change_df['date'] = percentage_change_df.index
percentage_change_df.head()

symbol,20MICRONS,21STCENMGM,3IINFOTECH,3MINDIA,3PLAND,5PAISA,63MOONS,A2ZINFRA,AAKASH,AARON,...,ZENITHEXPO,ZENSARTECH,ZENTEC,ZODIACLOTH,ZODJRDMKJ,ZOTA,ZUARI,ZUARIGLOB,ZYDUSWELL,date
date,,,,,,,,,,,,,,,,,,,,,
2011-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-01-03
2011-01-04,0.987138,NaN,0.995981,0.987066,1.006479,NaN,0.994161,1.026288,NaN,NaN,...,0.950378,0.994982,NaN,0.986612,1.005976,NaN,NaN,1.004672,1.040154,2011-01-04
2011-01-05,1.023580,NaN,0.997588,0.985948,0.984881,NaN,0.986863,1.031025,NaN,NaN,...,0.950378,1.022727,NaN,1.000000,1.003984,NaN,NaN,0.976499,1.022252,2011-01-05
2011-01-06,1.020364,NaN,0.990354,0.995080,0.967603,NaN,0.971648,0.998472,NaN,NaN,...,0.950378,1.049587,NaN,0.985496,0.964143,NaN,NaN,0.952856,1.025682,2011-01-06
2011-01-07,0.987138,NaN,0.944534,0.989260,0.958963,NaN,0.935717,0.961027,NaN,NaN,...,0.950378,1.024794,NaN,0.945581,0.992032,NaN,NaN,0.941743,1.011377,2011-01-07


### Convert the dataframe to the desired deserialised format

In [16]:
# Change the dataframe to the desired format for creating the bar race chart

percentage_change_melted_df = pd.melt(percentage_change_df, id_vars=['date'])
percentage_change_melted_df.head()

,date,symbol,value
0,2011-01-03,20MICRONS,NaN
1,2011-01-04,20MICRONS,0.987138
2,2011-01-05,20MICRONS,1.023580
3,2011-01-06,20MICRONS,1.020364
4,2011-01-07,20MICRONS,0.987138


In [17]:
# Save the deserialised dataframe to csv file

percentage_change_melted_df.to_csv("stock_price_percentage_changes_melted.csv")

This csv is then imported in tableau and create the bar race chart